# Program za prepoznavanje novčića

Pravim program za prepoznavanje novčića koristeći Python i OpenCV. Prvi korak je bio instalacija OpenCV biblioteke pomoću komande: 

pip install opencv -python

Nakon instalacije i postavljanja OpenCV-a, bilo je potrebno prikupiti slike novčića. Novčići su fotografisani na bijeloj pozadini radi lakšeg izdvajanja. Fotografisane slike su sačuvane u folderu 'obrada' gdje će se također nalaziti Python programi. Prvo slijedi objašnjenje za fajl 'obrada1.py'. Uvozimo dvije osnovne biblioteke u Pythonu za numeričko računanje i zadatke računarskog vida:

In [ ]:
import numpy as np
import cv2

Učitavanje slike, promjena veličine i pravljenje kopije koja će se koristiti kasnije:

In [ ]:
slika = cv2.imread("slika5.jpg")
slika = cv2.resize(slika, (640, 800))
s_kopija = slika.copy()

Sljedeći korak je primjena Gaussovog zamućenja na slici, što smanjuje šum i detalje slike kroz zaglađivanje slike. Parametri uključuju ulaznu sliku 'slika5.jpg', veličinu Gaussovog kernela (7, 7) za zamućenje, i standardnu devijaciju 3, koja određuje stupanj zamućenja primijenjenog u X i Y smjerovima:

In [ ]:
slika = cv2.GaussianBlur(slika, (7, 7), 3)

Slijedeći kod konvertuje sliku iz BGR prostora boja u nijanse sive. Zatim, primjenjuje binarno pragiranje na sliku u nijansama sive, postavljajući vrijednosti piksela ispod 138 na 0 i iznad 138 na 255, stvarajući binarnu sliku gdje su objekti od interesa istaknuti (Parametri se moraju prilagoditi za svaku specifičnu sliku):

In [ ]:
gray = cv2.cvtColor(slika, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 138, 255, cv2.THRESH_BINARY)

Ovaj segment koda koristi funkciju cv2.findContours() za identifikaciju kontura u binarnoj slici s pragom thresh, koristeći hijerarhijski režim pretraživanja i zadržavajući sve tačke konture bez aproksimacije. Zatim iterira kroz svaku pronađenu konturu, izračunava njenu površinu koristeći cv2.contourArea(), i pohranjuje indeks konture i njenu odgovarajuću površinu u rječniku nazvanom area:

In [ ]:
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
area = {}
for i in range(len(contours)):
    cnt = contours[i]
    ar = cv2.contourArea(cnt)
    area[i] = ar

Ovaj kod sortira rječnik area na osnovu površina kontura u silaznom redoslijedu, pretvarajući ga u numpy niz rez. Zatim, broji broj kontura čije su površine veće od 500 piksela identifikujući indekse gdje druga kolona rez (površine kontura) prelazi 500 koristeći np.argwhere(), i računajući njegov oblik kako bi odredio broj:

In [ ]:
srt = sorted(area.items(), key=lambda x: x[1], reverse=True)
rez = np.array(srt).astype("int")
num = np.argwhere(rez[:, 1] > 500).shape[0]

Ovaj kod iterira kroz sortirane konture, crtajući svaku konturu na kopiji slike s_kopija i pronalazeći minimalni ograđujući krug oko svake konture koristeći cv2.minEnclosingCircle(). Izračunava prečnik svakog novčića množenjem radijusa sa 2, dodaje prečnik i indeks konture na listu diameters, pružajući listu tuplova koji sadrže prečnik i indeks konture za svaki identifikovani novčić:

In [ ]:
diameters = []
for i in range(1, num):
    s_kopija = cv2.drawContours(s_kopija, contours, rez[i, 0], (0, 255, 0), 3)
    (x, y), radius = cv2.minEnclosingCircle(contours[rez[i, 0]])
    diameter = radius * 2  
    diameters.append((diameter, rez[i, 0]))

Ovaj segment koda sortira listu prečnika novčića u silaznom redoslijedu na osnovu njihovih vrijednosti prečnika. Zatim definiše vrijednosti novčića na osnovu sortiranih prečnika, pretpostavljajući da postoji tačno tri vrste novčića sa vrijednostima od 5.00, 1.00 i 0.05, redom. Na kraju, inicijalizira varijablu total_value za računanje ukupne vrijednosti novčića (pretpostavljajući da imamo tačno 3 vrste novčića):

In [ ]:
diameters.sort(reverse=True, key=lambda x: x[0])
coin_values = [5.00, 1.00, 0.05]  
total_value = 0



Ovaj kod iterira kroz sortiranu listu prečnika kovanica i njihovih odgovarajućih indeksa kontura. On dodjeljuje vrijednost svakoj kovanici na osnovu njene pozicije u sortiranoj listi, ciklično prolazeći kroz unaprijed definirane vrijednosti kovanica ako se otkrije više kovanica nego što je definirano vrijednosti. Ukupna vrijednost kovanica se izračunava inkrementalno. On ispisuje prečnik i dodijeljenu vrijednost za svaku prepoznatu kovanicu i anotira sliku vrijednošću kovanice koristeći OpenCV-ovu funkciju cv2.putText():


In [ ]:
for idx, (diameter, contour_idx) in enumerate(diameters):
    coin_value = coin_values[idx % len(coin_values)]  
    total_value += coin_value
    
    
    print(f"Recognized coin {idx+1}: Diameter = {diameter:.2f} pixels, Value = {coin_value} KM")
    
    
    (x, y), radius = cv2.minEnclosingCircle(contours[contour_idx])
    center = (int(x), int(y))
    cv2.putText(s_kopija, f"{coin_value} KM", center, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

Ispis ukupne vrijednosti i prikaz konačnih rezultata:


In [ ]:
print("Total value:", total_value, "KM")

cv2.imshow("rezultat", s_kopija)
cv2.waitKey(0)
cv2.destroyAllWindows()

Za datoteku obrada.py, korišten je isti princip, samo što na slici ima više vrsta različitih kovanica. U ovom primjeru možemo primijetiti kako su parametri morali biti prilagođeni:

In [ ]:
ret, thresh = cv2.threshold(gray, 142, 255, cv2.THRESH_BINARY)

Za datoteku 'obrada2.py': Ovaj dio koda iterira kroz konture kovanica detektiranih na slici. Za svaku konturu izračunava prečnik na osnovu minimalnog obuhvatnog kruga. U zavisnosti od raspona prečnika, dodjeljuje odgovarajuću vrijednost kovanice. Konture s prepoznatim vrijednostima kovanica grupiraju se zajedno na osnovu dodijeljenih vrijednosti u dictionaryju coin_groups:

In [ ]:
coin_groups = {}

for contour in contours:
    (x, y), radius = cv2.minEnclosingCircle(contour)
    diameter = radius * 2  

    if 75 <= diameter <= 76:
        coin_value = 0.05
    elif 88 <= diameter <= 89:
        coin_value = 0.1
    elif 90 <= diameter <= 91:
        coin_value = 0.2
    elif 95 <= diameter <= 97.8:
        coin_value = 1
    elif 98 <= diameter <= 99:
        coin_value = 0.5
    elif 100 <= diameter <=120:
        coin_value = 5
    else:
        coin_value = None

    if coin_value is not None:
        coin_groups.setdefault(coin_value, []).append(contour)

Rječnik coin_values mapira vrijednosti kovanica na njihove odgovarajuće apoene u valuti KM (Konvertibilna Marka). Svaki ključ predstavlja vrijednost kovanice (u KM), dok odgovarajuća vrijednost predstavlja string koji označava apoen kovanice:

In [ ]:
coin_values = {
    5.00: "5 KM",
    1.00: "1 KM",
    0.50: "0.50 KM",
    0.20: "0.20 KM",
    0.10: "0.10 KM",
    0.05: "0.05 KM"
}

Rječnik grouped_coins organizira detektirane kovanice na osnovu njihovih vrijednosti. On iterira kroz svaku vrijednost kovanice u rječniku coin_groups i dodjeljuje odgovarajući apoen iz rječnika coin_values. Konture kovanica koje pripadaju svakom apoenu zatim se grupiraju zajedno pod njihovim odgovarajućim apoenom u rječniku grouped_coins:

In [ ]:
grouped_coins = {}
for coin_value, group in coin_groups.items():
    grouped_coins.setdefault(coin_values[coin_value], []).extend(group)

Ovaj dio koda iterira kroz rječnik grouped_coins, ispisujući grupu apoena i broj kovanica u svakoj grupi. Zatim izračunava ukupnu vrijednost svih detektiranih kovanica u KM (Konvertibilna Marka) zbrajanjem proizvoda svake vrijednosti kovanice i broja kovanica u toj grupi vrijednosti. Na kraju, ispisuje ukupnu vrijednost u KM:

In [ ]:
for group, coins in grouped_coins.items():
    print(f"Group: {group}, Count: {len(coins)}")


total_value = sum(coin_value * len(coins) for coin_value, coins in coin_groups.items())
print("Total value in KM:", total_value)

Ovaj kod iterira kroz stavke u rječniku grouped_coins. Za svaku grupu kovanica, generira slučajnu boju koristeći NumPy i crta konture kovanica na slici s_kopija koristeći OpenCV-ovu funkciju cv2.drawContours(). Svaka grupa kovanica se crta s konturama iste boje kako bi se vizualno razlikovale na slici:

In [ ]:
for group, coins in grouped_coins.items():
    color = np.random.randint(0, 255, size=3).tolist() 
    for contour in coins:
        s_kopija = cv2.drawContours(s_kopija, [contour], -1, color, 3)